In [1]:
from pathlib import Path

from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset/'
sequence = '00'
sam_labels_path = Path.cwd().joinpath('pipeline', 'vfm-labels', 'sam', sequence)

kitti = KittiDataset(dataset_path, sequence, sam_labels_path)

In [2]:
from mapping.simple_mapping import SimpleMapping

sm = SimpleMapping(kitti)
labeled_pcds, colored_pcds, point_labels_dict = sm.get_combined_labeled_point_clouds('cam2', 20, 23)

INFO:simple_mapping.SimpleMapping:Image 20 processed
INFO:simple_mapping.SimpleMapping:Image 21 processed
INFO:simple_mapping.SimpleMapping:Image 22 processed


In [3]:
print(len(point_labels_dict))

279921


In [4]:
point40 = point_labels_dict[40]
print(point40)

[<mapping.label.Label object at 0x29df7c040>, <mapping.label.Label object at 0x2bd07d070>, <mapping.label.Label object at 0x2cdff6100>]


In [6]:
label0 = point40[0]
print(label0.image_index)

20


In [7]:
label0.instance_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [8]:
label1 = point40[1]
print(label1.image_index)

21


In [9]:
label1.instance_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [10]:
label2 = point40[2]
print(label2.image_index)

22


In [11]:
label2.instance_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])